In [ ]:
%pip install requests beautifulsoup4

In [ ]:
import os
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


task 1

In [ ]:
def download_page_content(url, filename, directory):
  """Скачивает содержимое страницы и сохраняет его в файл."""

  inner_html_code = str(urlopen(url).read(), 'utf-8')
  inner_soup = BeautifulSoup(inner_html_code, "html.parser")

  os.makedirs(os.path.join("man_pages", directory), exist_ok=True)
  with open(os.path.join("man_pages", directory, filename), "w", encoding='utf-8') as f:
    f.write(inner_soup.get_text())

base_url = "https://man7.org/linux/man-pages/dir_all_by_section.html"

inner_html_code = str(urlopen(base_url).read(), 'utf-8')
inner_soup = BeautifulSoup(inner_html_code, "html.parser")

links = inner_soup.find_all('a', href=True)

section_links = [link['href'] for link in links if link['href'].startswith('#man')]
# print(section_links)

pages = []
parts = base_url.rsplit('/', 1)
base_url = base_url = '/'.join(parts[:-1])
for section_link in section_links:
  section_number = section_link[4:]
  pre_blocks = inner_soup.find_all('pre')
  # print(pre_blocks)
  for pre_block in pre_blocks:
    # print(pre_block)
    function_links = pre_block.find_all('a', href=True)
    for link in function_links:
      if f'man{section_number}' in link['href']:
          href = link['href']
          # print(href)
          if href.startswith('./'):
            href = href[1:]
            # print(href)
          full_url = base_url + href
          pages.append(full_url)
# print(pages)
for url in tqdm(pages, desc=f"Скачивание мануалов"):
  parts = url.split('/')
  name = parts[-1].split('.')
  if len(name) > 2:
    name = '.'.join(name[:-2])
  else:
    name = name[0]
  filename_extension = parts[-1].split('.')[-2]
  if filename_extension == name:
    filename = f"{name}"
  else:
    filename = f"{name}({filename_extension})"

  directory = parts[-2] if parts[-2] != 'man-pages' else ''
  if name.startswith('.'):
    continue
  if directory >= 'man5':
    download_page_content(url, filename, directory)

print("Содержимое страниц скачано в папку 'man_pages'.")

Скачивание мануалов: 100%|██████████| 13424/13424 [12:56<00:00, 17.29it/s]

Содержимое страниц скачано в папку 'man_pages'.


Встретилас ссылка, которая не откравается (начинается с "."), поэтому я просто дополнилаё код, чтобы не скачивать все занаво.

task 2:  Написать скрипт для загрузки всех файлов в базу векторов (выбрать любимую/любую)


In [ ]:
%pip install sentence-transformers faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 40.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyli

In [ ]:
%pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [ ]:
from sentence_transformers import SentenceTransformer, util
from faiss import IndexFlatL2, IndexIVFFlat, write_index
from datasets import Dataset
import faiss
import torch
import numpy as np

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
def texts_list(files_path):
    data = []
    for root, dirs, files in os.walk(files_path): #рекурсия
        if ".ipynb_checkpoints" in dirs:
          dirs.remove(".ipynb_checkpoints")
        for filename in files:
          filepath = os.path.join(root, filename)
          with open(filepath, "r", encoding="utf-8") as f:
            text = f.read()
            title = filename[:-4]
            data.append({'title': title, 'text': text})

        if root != files_path:
            folder_index = root.split('/')[-1].split('man')[-1]
            print(f"Файлы в папке man{folder_index} готовы к преобразованию в векторы")

    dataset = pd.DataFrame(data)
    return dataset

files_path = "/content/drive/MyDrive/man_pages"
output_path = "/content/drive/MyDrive/manuals_dop_files"
dataset = texts_list(files_path)
dataset.to_csv(os.path.join(output_path, "dataset.csv"), index=False)

Файлы в папке man0 готовы к преобразованию в векторы
Файлы в папке man1 готовы к преобразованию в векторы
Файлы в папке man4 готовы к преобразованию в векторы
Файлы в папке man5 готовы к преобразованию в векторы
Файлы в папке man6 готовы к преобразованию в векторы
Файлы в папке man7 готовы к преобразованию в векторы
Файлы в папке man8 готовы к преобразованию в векторы
Файлы в папке man2 готовы к преобразованию в векторы
Файлы в папке man3 готовы к преобразованию в векторы


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/manuals_dop_files/dataset.csv')
df.head(3)

,title,text
0,arpa_inet.h,\n\n\n\n\n\narpa_inet.h(0p) - Linux manual pag...
1,complex.h,\n\n\n\n\n\ncomplex.h(0p) - Linux manual page\...
2,assert.h,\n\n\n\n\n\nassert.h(0p) - Linux manual page\n...


In [ ]:
data = Dataset.from_pandas(df)
data

Dataset({
    features: ['title', 'text'],
    num_rows: 13423
})

In [ ]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer_man = AutoTokenizer.from_pretrained(model_ckpt)
model_man = AutoModel.from_pretrained(model_ckpt)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_man.to(device)

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_

In [ ]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]
def get_embeddings(text_list):
    encoded_input = tokenizer_man(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model_man(**encoded_input)
    return cls_pooling(model_output)

In [ ]:
embeddings_dataset = data.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]}
)

Map:   0%|          | 0/13423 [00:00<?, ? examples/s]

In [ ]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/14 [00:00<?, ?it/s]

Dataset({
    features: ['title', 'text', 'embeddings'],
    num_rows: 13423
})

In [ ]:
embeddings_dataset

Dataset({
    features: ['title', 'text', 'embeddings'],
    num_rows: 13423
})

In [ ]:
output_path = "/content/drive/MyDrive/manuals_dop_files"
embeddings_dataset.to_csv(os.path.join(output_path, "embeddings_dataset.csv"), index=False)

Creating CSV from Arrow format:   0%|          | 0/14 [00:00<?, ?ba/s]

289037014

In [ ]:
embeddings_dataset = pd.read_csv('/content/drive/MyDrive/manuals_dop_files/embeddings_dataset.csv')
embeddings_dataset.head(3)

,title,text,embeddings
0,arpa_inet.h,\n\n\n\n\n\narpa_inet.h(0p) - Linux manual pag...,[-7.75886625e-02 -3.73882443e-01 -1.13922648e-...
1,complex.h,\n\n\n\n\n\ncomplex.h(0p) - Linux manual page\...,[-3.27646732e-01 -4.78862137e-01 -2.06932962e-...
2,assert.h,\n\n\n\n\n\nassert.h(0p) - Linux manual page\n...,[-1.08496159e-01 -4.84370410e-01 -9.24604088e-...


task 3:  Написать простой скрипт для semantic search по базе

In [ ]:
query = "How to get current directory in Linux"
query_embedding = get_embeddings([query]).cpu().detach().numpy()
query_embedding.shape

(1, 768)

In [ ]:
embeddings_dataset

Dataset({
    features: ['title', 'text', 'embeddings'],
    num_rows: 13423
})

In [ ]:
def semantic_search(query, k):
    query_embedding = get_embeddings([query]).cpu().detach().numpy()
    scores, samples = embeddings_dataset.get_nearest_examples("embeddings", query_embedding, k)

    samples_df = pd.DataFrame.from_dict(samples)
    samples_df["scores"] = scores
    samples_df.sort_values("scores", ascending=False, inplace=True)

    for _, row in samples_df.iterrows():
        print(f"SCORE: {row.scores}")
        print(f"TITLE: {row.title}")
        print(f"TEXT: {row.text}")
        print("=" * 50)
        print()

k = 1
query = "How to get current directory in Linux"
semantic_search(query, k)

SCORE: 26.486404418945312
TITLE: pw
TEXT: 





pwd(1) - Linux manual page











man7.org > Linux > man-pages



Linux/UNIX system programming training






pwd(1) — Linux manual page




NAME | SYNOPSIS | DESCRIPTION | AUTHOR | REPORTING BUGS | COPYRIGHT | SEE ALSO | COLOPHON















 




PWD(1)                        User Commands                       PWD(1)

NAME          top
       pwd - print name of current/working directory

SYNOPSIS          top
       pwd [OPTION]...

DESCRIPTION          top
       Print the full filename of the current working directory.

       -L, --logical
              use PWD from environment, even if it contains symlinks

       -P, --physical
              avoid all symlinks

       --help display this help and exit

       --version
              output version information and exit

       If no option is specified, -P is assumed.

       NOTE: your shell may have its own version of pwd, which usually
       supersedes the version desc

3* Chat-bot

In [ ]:
%pip install -q accelerate bitsandbytes


In [ ]:
%pip install accelerate

In [ ]:
def search(query: str, k: int = 3 ):
    query_embedding = get_embeddings([query]).cpu().detach().numpy()
    scores, retrieved_examples = embeddings_dataset.get_nearest_examples(
        "embeddings", query_embedding,
        k=k
    )
    return scores, retrieved_examples

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
model_name = "rainjay/Meta-Llama-3-8B-Instruct-4bit"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=bnb_config
)
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:167: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

In [ ]:
SYS_PROMPT = """You are an assistant for answering questions.
You are given the extracted parts of a long document and a question. Provide a conversational answer.
If you don't know the answer, just say "I do not know." Don't make up an answer."""

In [ ]:
def format_prompt(prompt,retrieved_documents,k):
  """using the retrieved documents we will prompt the model to generate our responses"""
  PROMPT = f"Question:{prompt}\nContext:"
  for idx in range(k) :
    PROMPT+= f"{retrieved_documents['text'][idx]}\n"
  return PROMPT

def generate(formatted_prompt):
    formatted_prompt = formatted_prompt[:2000]
    messages = [{"role":"system","content":SYS_PROMPT},{"role":"user","content":formatted_prompt}]
    input_ids = tokenizer.apply_chat_template(
        messages,
        truncation=True,
        add_generation_prompt=True,
        return_tensors="pt",
        max_length=2000
    ).to(model.device)
    attention_mask = torch.ones_like(input_ids)
    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=1024,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)

def rag_chatbot(prompt:str,k:int=2):
  scores , retrieved_documents = search(prompt, k)
  formatted_prompt = format_prompt(prompt,retrieved_documents,k)
  return generate(formatted_prompt)


In [ ]:
rag_chatbot("How to get current directory in Linux", k = 2)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"To get the current directory in Linux, you can use the `pwd` command. You can simply type `pwd` in the terminal and press Enter, and it will print the full path of the current working directory.\n\nIf you want to include the logical directory path, which may include symbolic links, you can use the `-L` or `--logical` option. If you want to avoid symbolic links and get the physical directory path, you can use the `-P` or `--physical` option.\n\nFor example, if you're in a directory that has a symbolic link to another directory, using `pwd` without any options will show you the logical path, while using `pwd -P` will show you the physical path.\n\nNote that your shell may also have its own version of the `pwd` command, which may support additional options."

2* DAG